# 流式输出

langchain的Runnable对象的8种输出方法中，有一半与流式输出有关：
- stream
- astream
- astream_events
- astream_log

## 准备

In [239]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

## stream

### 示例：Prompt+LLM

In [240]:
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_openai import ChatOpenAI, OpenAI

llm = OpenAI().bind(seed=42)
prompt = PromptTemplate.from_template("讲个关于{topic}的笑话")

### 流式打印

In [241]:
for chunk in (prompt| llm).stream({"topic": "bears"}):
    print(chunk, end="_", flush=True)



_为_什_么_熊_总_是_穿_着_围_裙_？_因_为_它_们_喜_欢_吃_烤_饼_！__

## 将流解析为字符串

### AIMessageChunk

In [110]:
llm = ChatOpenAI().bind(seed=42)
prompt = ChatPromptTemplate.from_template("讲个关于{topic}的笑话")
chain = prompt | llm

In [111]:
chunks = []
for chunk in chain.stream({"topic": "bears"}):
    chunks.append(chunk)
    print(chunk.content, end="_", flush=True)

_为_什_么_熊_喜_欢_在_雨_天_打_开_电_视_？
_因_为_他_们_喜_欢_看_“_熊_熊_有_词_”_！__

实际上每个 chunk 长这样：

In [112]:
chunks[0:4]

[AIMessageChunk(content=''),
 AIMessageChunk(content='为'),
 AIMessageChunk(content='什'),
 AIMessageChunk(content='么')]

### StrOutputParser

In [113]:
from langchain_core.output_parsers import StrOutputParser

chunks = []
json_chain = prompt | model | StrOutputParser()
for chunk in json_chain.stream({"topic": "bears"}):
    chunks.append(chunk)
    print(chunk, end="_", flush=True)

_为_什_么_熊_喜_欢_在_雨_天_打_开_电_视_？
_因_为_他_们_喜_欢_看_“_熊_熊_有_词_”_！__

In [114]:
chunks[0:4]

['', '为', '什', '么']

## 流处理技巧

### 生成一个流内容

In [181]:
from typing import Iterator, List

from langchain.prompts.chat import ChatPromptTemplate
from langchain_openai import OpenAI

prompt = ChatPromptTemplate.from_template(
    "生成与{animal}相似的5种动物名称，格式要求：生成一个名称列表字符串，逗号间隔。"
)
model = OpenAI(temperature=0).bind(seed=42)

str_chain = prompt | model

In [183]:
for chunk in str_chain.stream({"animal": "猫"}):
    print(chunk, end="_", flush=True)



_狮_子_,_ 老_虎_,_ 豹_子_,_ 豹_猫_,_ 豹_狗__

In [184]:
str_chain.invoke({"animal": "猫"})

'\n\n狮子, 老虎, 豹子, 豹猫, 豹狗'

### 处理流式输入

In [185]:
# This is a custom parser that splits an iterator of llm tokens
# into a list of strings separated by commas
def split_into_list(input: Iterator[str]) -> Iterator[List[str]]:
    # hold partial input until we get a comma
    buffer = ""
    for chunk in input:
        # add current chunk to buffer
        buffer += chunk
        # while there are commas in the buffer
        while "," in buffer:
            # split buffer on comma
            comma_index = buffer.index(",")
            # yield everything before the comma
            yield [buffer[:comma_index].strip()]
            # save the rest for the next iteration
            buffer = buffer[comma_index + 1 :]
    # yield the last chunk
    yield [buffer.strip()]

In [186]:
list_chain = str_chain | split_into_list

In [190]:
for chunk in list_chain.stream({"animal": "猫"}):
    print(chunk, end = "_", flush=True)

['狮子']_['老虎']_['豹子']_['豹猫']_['豹狗']_

In [189]:
list_chain.invoke({"animal": "猫"})

['狮子', '老虎', '豹子', '豹猫', '豹狗']

### 实现一个异步版本

In [199]:
from typing import AsyncIterator

async def asplit_into_list(
    input: AsyncIterator[str],
) -> AsyncIterator[List[str]]:  # async def
    buffer = ""
    async for (
        chunk
    ) in input:  # `input` is a `async_generator` object, so use `async for`
        buffer += chunk
        while "," in buffer:
            comma_index = buffer.index(",")
            yield [buffer[:comma_index].strip()]
            buffer = buffer[comma_index + 1 :]
    yield [buffer.strip()]

list_chain = str_chain | asplit_into_list

In [205]:
async for chunk in list_chain.astream({"animal": "猫"}):
    print(chunk, end = "_", flush=True)

['狮子']_['老虎']_['豹子']_['豹猫']_['豹狗']_

In [203]:
await list_chain.ainvoke({"animal": "猫"})

['狮子', '老虎', '豹子', '豹猫', '豹狗']

## 将流解析为字典

### 将JSON流输出解析为字典

<div class="alert-warning" style="padding: 5px">
<b>注意：</b>
    
**JsonOutputParser** 会输出一个字典结构。

但为了随时获得完整的JSON结构，**JsonOutputParser** 会随时根据已收到的字符串构造最新可用的字典结构。<br>
这与 **StrOutputParser** 非常不同！
</div>

In [130]:
from langchain_core.output_parsers import JsonOutputParser

chain = model | JsonOutputParser()

final_json = {}
for chunk in chain.stream('用JSON格式输出一个包含若干亚洲国家的名称和人口数量，国家名称使用中文'):
    final_json = chunk
    print(chunk, flush=True)

{}
{'countries': []}
{'countries': [{}]}
{'countries': [{'name': ''}]}
{'countries': [{'name': '中国'}]}
{'countries': [{'name': '中国', 'population': 140}]}
{'countries': [{'name': '中国', 'population': 140005}]}
{'countries': [{'name': '中国', 'population': 140005000}]}
{'countries': [{'name': '中国', 'population': 1400050000}]}
{'countries': [{'name': '中国', 'population': 1400050000}, {}]}
{'countries': [{'name': '中国', 'population': 1400050000}, {'name': ''}]}
{'countries': [{'name': '中国', 'population': 1400050000}, {'name': '印'}]}
{'countries': [{'name': '中国', 'population': 1400050000}, {'name': '印度'}]}
{'countries': [{'name': '中国', 'population': 1400050000}, {'name': '印度', 'population': 136}]}
{'countries': [{'name': '中国', 'population': 1400050000}, {'name': '印度', 'population': 136600}]}
{'countries': [{'name': '中国', 'population': 1400050000}, {'name': '印度', 'population': 136600000}]}
{'countries': [{'name': '中国', 'population': 1400050000}, {'name': '印度', 'population': 1366000000}]}
{'countr

In [128]:
# final_json 是一个字典
final_json

{'countries': [{'name': 'China', 'population': 1439323776},
  {'name': 'India', 'population': 1380004385},
  {'name': 'Indonesia', 'population': 273523615},
  {'name': 'Pakistan', 'population': 220892340},
  {'name': 'Bangladesh', 'population': 164689383},
  {'name': 'Japan', 'population': 125360000},
  {'name': 'Philippines', 'population': 109581078},
  {'name': 'Vietnam', 'population': 97338579}]}

### 将JSON流作为输入处理

In [135]:
def _extract_country_names_streaming(input_stream):
    """A function that operates on input streams."""
    country_names_so_far = set()

    for input in input_stream:
        # 跳过尚未构建字典的情况
        if not isinstance(input, dict):
            continue

        # 跳过尚未发现countries键的情况
        if "countries" not in input:
            continue

        # 获取countries
        countries = input["countries"]

        # 跳过countries还没构建为列表的情况
        if not isinstance(countries, list):
            continue

        # 此时字典结构已经构建完毕，可以枚举每个元素，用来构建生成器
        for country in countries:
            name = country.get("name")
            population = country.get("population")
            if not name:
                continue
            # 当population键已经生成，意味着name已经有完整的值
            if not population:
                continue
            if name not in country_names_so_far:
                yield name
                country_names_so_far.add(name)

# 构建一个链，其中_extract_country_names_streaming被自动转换为 RunnableLambda
chain = model | JsonOutputParser() | _extract_country_names_streaming

for chunk in chain.stream('用JSON格式输出一个包含20个亚洲国家的名称和人口数量，国家名称使用中文'):
    print(chunk, end="_", flush=True)

中国_印度_印度尼西亚_巴基斯坦_孟加拉国_日本_菲律宾_越南_土耳其_伊朗_泰国_缅甸_韩国_伊拉克_阿富汗_沙特阿拉伯_乌兹别克斯坦_马来西亚_也门_朝鲜_

## astream_event

### Runnable的事件流

In [136]:
events = []
async for event in model.astream_events("hello", version="v1"):
    events.append(event)

In [144]:
events[0:3]

[{'event': 'on_chat_model_start',
  'run_id': '555f4b4f-9455-43d9-9025-46b91c0ad80f',
  'name': 'ChatOpenAI',
  'tags': [],
  'metadata': {},
  'data': {'input': 'hello'}},
 {'event': 'on_chat_model_stream',
  'run_id': '555f4b4f-9455-43d9-9025-46b91c0ad80f',
  'tags': [],
  'metadata': {},
  'name': 'ChatOpenAI',
  'data': {'chunk': AIMessageChunk(content='')}},
 {'event': 'on_chat_model_stream',
  'run_id': '555f4b4f-9455-43d9-9025-46b91c0ad80f',
  'tags': [],
  'metadata': {},
  'name': 'ChatOpenAI',
  'data': {'chunk': AIMessageChunk(content='Hello')}}]

### chain的事件流

In [154]:
chain = (
    model | JsonOutputParser()
)  # Due to a bug in older versions of Langchain, JsonOutputParser did not stream results from some models

events = [
    event
    async for event in chain.astream_events(
        """
        用JSON格式输出包含20个亚洲国家名称和人口数量的数据；
        格式要求：
        1. 每个国家应当有 name 和 population 两个键
        2. 国家名称使用中文
        """,
        version="v1",
    )
]

<div class="alert-info">
<b>注意</b>
    
由于OpenAI和JsonOutputParser都支持stream，因此可以看到on_chat_model_stream和on_parser_stream交替出现。
</div>

In [159]:
events[30:40]

[{'event': 'on_chat_model_stream',
  'name': 'ChatOpenAI',
  'run_id': 'e4bd6e4a-d81b-44b4-b9b1-d565498cef46',
  'tags': ['seq:step:1'],
  'metadata': {},
  'data': {'chunk': AIMessageChunk(content=' "')}},
 {'event': 'on_chat_model_stream',
  'name': 'ChatOpenAI',
  'run_id': 'e4bd6e4a-d81b-44b4-b9b1-d565498cef46',
  'tags': ['seq:step:1'],
  'metadata': {},
  'data': {'chunk': AIMessageChunk(content='population')}},
 {'event': 'on_chat_model_stream',
  'name': 'ChatOpenAI',
  'run_id': 'e4bd6e4a-d81b-44b4-b9b1-d565498cef46',
  'tags': ['seq:step:1'],
  'metadata': {},
  'data': {'chunk': AIMessageChunk(content='":')}},
 {'event': 'on_chat_model_stream',
  'name': 'ChatOpenAI',
  'run_id': 'e4bd6e4a-d81b-44b4-b9b1-d565498cef46',
  'tags': ['seq:step:1'],
  'metadata': {},
  'data': {'chunk': AIMessageChunk(content=' ')}},
 {'event': 'on_parser_stream',
  'name': 'JsonOutputParser',
  'run_id': '86750e5b-8182-4209-854b-9090ac9fec20',
  'tags': ['seq:step:2'],
  'metadata': {},
  'data'

### RAG的事件流

下面是一个更复杂的RAG链的例子。<br>
你可以在创建 Chain 的时候为其命名，并在执行时按照名称过滤和打印每个Runnable的日志。

In [145]:
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings

template = """基于以下内容回答问题：
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

vectorstore = Chroma.from_texts(
    ["张朝阳在搜狐公司工作"], embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

retrieval_chain = (
    {
        "context": retriever.with_config(run_name="本地检索"),
        "question": RunnablePassthrough(),
    }
    | prompt
    | model.with_config(run_name="LLM")
    | StrOutputParser()
)

<div class="alert-info">
<b>注意</b>

如果需要打印日志，你可以根据name、tags或event等类型来过滤。

简单的方法是在调用astream_events的时候就过滤，可以用的方法包括：
- include_names/include_tags/include_types 用于包含
- exclude_names/exclue_tags/exclude_types 用于排除
</div>

In [150]:
events = []
async for event in retrieval_chain.astream_events(
    "张朝阳在哪里上班？", version="v1", include_names=["本地检索", "LLM"]
):
    events.append(event)
    kind = event["event"]
    if kind == "on_chat_model_stream":
        print(event["data"]["chunk"].content, end="_")
    elif kind in {"on_chat_model_start"}:
        print("\nLLM流开始:")
    elif kind in {"on_chat_model_end"}:
        print("\nLLM流结束.")
    elif kind == "on_retriever_end":
        print("\nRAG检索结果:")
        print(event["data"]["output"]["documents"])
    elif kind == "on_tool_end":
        print(f"工具调用结束: {event['name']}")
    else:
        pass

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


--
RAG检索结果:
[Document(page_content='张朝阳在搜狐公司工作'), Document(page_content='张朝阳在搜狐公司工作')]

LLM流开始:
_张_朝_阳_在_搜_狐_公司_工_作_。__
LLM流结束.


In [152]:
events[0:-1]

[{'event': 'on_chain_start',
  'run_id': 'dee1e825-cf5c-46ed-a703-84263b1ee598',
  'name': 'RunnableSequence',
  'tags': [],
  'metadata': {},
  'data': {'input': '张朝阳在哪里上班？'}},
 {'event': 'on_chain_start',
  'name': 'RunnableParallel<context,question>',
  'run_id': 'e7db3803-183a-4b5a-8fe7-f4e4a695eaf9',
  'tags': ['seq:step:1'],
  'metadata': {},
  'data': {}},
 {'event': 'on_retriever_start',
  'name': '本地检索',
  'run_id': '20408b6e-8e81-4b54-9cdf-f8f43dda85ce',
  'tags': ['map:key:context', 'Chroma', 'OpenAIEmbeddings'],
  'metadata': {},
  'data': {'input': {'query': '张朝阳在哪里上班？'}}},
 {'event': 'on_chain_start',
  'name': 'RunnablePassthrough',
  'run_id': 'eeb62b00-fffe-4b86-a88b-b77d3967ff9e',
  'tags': ['map:key:question'],
  'metadata': {},
  'data': {}},
 {'event': 'on_chain_stream',
  'name': 'RunnablePassthrough',
  'run_id': 'eeb62b00-fffe-4b86-a88b-b77d3967ff9e',
  'tags': ['map:key:question'],
  'metadata': {},
  'data': {'chunk': '张朝阳在哪里上班？'}},
 {'event': 'on_chain_stream

### 在工具中支持事件流

<div class="alert-warning" style="padding: 5px">
    <b>注意：</b><br>
如果工具中使用astream_events，必须传播 callbacks 参数！
</div>

In [171]:
from langchain_core.runnables import RunnableLambda
from langchain_core.tools import tool

def reverse_word(word: str):
    return word[::-1]

reverse_word = RunnableLambda(reverse_word)

@tool
def bad_tool(word: str):
    """工具定义中没有传播 callbacks"""
    return reverse_word.invoke(word)

async for event in bad_tool.astream_events("hello", version="v1"):
    print(event)

{'event': 'on_tool_start', 'run_id': '154f57ea-8b3c-4260-8a64-6d85cc1edb46', 'name': 'bad_tool', 'tags': [], 'metadata': {}, 'data': {'input': 'hello'}}
{'event': 'on_tool_stream', 'run_id': '154f57ea-8b3c-4260-8a64-6d85cc1edb46', 'tags': [], 'metadata': {}, 'name': 'bad_tool', 'data': {'chunk': 'olleh'}}
{'event': 'on_tool_end', 'name': 'bad_tool', 'run_id': '154f57ea-8b3c-4260-8a64-6d85cc1edb46', 'tags': [], 'metadata': {}, 'data': {'output': 'olleh'}}


如果没有正确传播 **callbacks** 入参，则不会生成 **reverse_word** 相关的事件流。<br>
注意上面代码与下面代码的输出差异：**name** 为 **reverse_word** 的两行事件流日志。

In [172]:
@tool
def correct_tool(word: str, callbacks):
    """工具定义中正确传播了 callbacks 入参"""
    return reverse_word.invoke(word, {"callbacks": callbacks})

async for event in correct_tool.astream_events("hello", version="v1"):
    print(event)

{'event': 'on_tool_start', 'run_id': '5c984fcd-1604-4bca-a3b4-b00dacc86555', 'name': 'correct_tool', 'tags': [], 'metadata': {}, 'data': {'input': 'hello'}}
{'event': 'on_chain_start', 'name': 'reverse_word', 'run_id': '6390ba50-8c0d-494b-bb92-6c18ea7061b6', 'tags': [], 'metadata': {}, 'data': {'input': 'hello'}}
{'event': 'on_chain_end', 'name': 'reverse_word', 'run_id': '6390ba50-8c0d-494b-bb92-6c18ea7061b6', 'tags': [], 'metadata': {}, 'data': {'input': 'hello', 'output': 'olleh'}}
{'event': 'on_tool_stream', 'run_id': '5c984fcd-1604-4bca-a3b4-b00dacc86555', 'tags': [], 'metadata': {}, 'name': 'correct_tool', 'data': {'chunk': 'olleh'}}
{'event': 'on_tool_end', 'name': 'correct_tool', 'run_id': '5c984fcd-1604-4bca-a3b4-b00dacc86555', 'tags': [], 'metadata': {}, 'data': {'output': 'olleh'}}


## 智能体中的流式输出

### 找猫的游戏

In [277]:
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain.prompts import ChatPromptTemplate
from langchain.tools import tool
from langchain_core.callbacks import Callbacks
from langchain_openai import ChatOpenAI

In [278]:
model = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0, streaming=True).bind(seed=42)

In [279]:
import random

@tool
async def where_is_cat_hiding(idea: str) -> str:
    """从这些地方选择猫躲藏的地方"""
    return random.choice(["在床底吗？", "在书架上吗？"])

In [280]:
@tool
async def get_items(place: str) -> str:
    """使用这个工具去仔细查看猫躲藏的地方"""
    if "床底" in place:
        return "拖鞋、袜子和脏东西，还有一只猫"
    if "书架" in place:
        return "书、铅笔还有绘本，但没发现猫"
    else:  # if the agent decides to ask about a different place
        return "算了，找不到"

In [281]:
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-tools-agent")
# print(prompt.messages) -- to see the prompt
tools = [get_items, where_is_cat_hiding]
agent = create_openai_tools_agent(
    model.with_config({"tags": ["agent_llm"]}),
    tools,
    prompt
)

In [282]:
agent_executor = AgentExecutor(agent=agent, tools=tools).with_config({"run_name": "Agent"})

### ainvoke

In [284]:
await agent_executor.ainvoke({"input": "猫呢?"})

{'input': '猫呢?', 'output': '猫就躲在床底下！'}

### astream

In [283]:
async for chunk in agent_executor.astream({"input": "猫呢?"}):
    # Agent Action
    if "actions" in chunk:
        for action in chunk["actions"]:
            print(f"Calling Tool: `{action.tool}` with input `{action.tool_input}`")
    # Observation
    elif "steps" in chunk:
        for step in chunk["steps"]:
            print(f"Tool Result: `{step.observation}`")
    # Final result
    elif "output" in chunk:
        print(f'Final Output: {chunk["output"]}')
    else:
        raise ValueError()
    print("-" * 80)

Calling Tool: `where_is_cat_hiding` with input `{'idea': '在书架上'}`
--------------------------------------------------------------------------------
Tool Result: `在床底吗？`
--------------------------------------------------------------------------------
Calling Tool: `where_is_cat_hiding` with input `{'idea': '在床底'}`
--------------------------------------------------------------------------------
Tool Result: `在床底吗？`
--------------------------------------------------------------------------------
Calling Tool: `where_is_cat_hiding` with input `{'idea': '在床底'}`
--------------------------------------------------------------------------------
Tool Result: `在床底吗？`
--------------------------------------------------------------------------------
Calling Tool: `where_is_cat_hiding` with input `{'idea': '在床底'}`
--------------------------------------------------------------------------------
Tool Result: `在床底吗？`
--------------------------------------------------------------------------------
Calling

### astream_event

In [290]:
events = []
async for event in agent_executor.astream_events({"input": "猫呢?"}, version="v1"):
    events.append(event)
    kind = event["event"]
    if kind == "on_chat_model_stream":
        print(event["data"]["chunk"].content, end="_")
    elif kind == "on_retriever_end":
        print("\nRAG检索结果:")
        print(event["data"]["output"]["documents"])
    elif kind == "on_tool_end":
        print("\n工具调用结束:")
        print(event['name'])
        print(event['data'])
    else:
        pass

__________
工具调用结束:
where_is_cat_hiding
{'input': {'idea': '在书架上'}, 'output': '在书架上吗？'}
_让_我_去_书_架_上_找_找_看_。_________
工具调用结束:
get_items
{'input': {'place': '书架'}, 'output': '书、铅笔还有绘本，但没发现猫'}
_在_书_架_上_没有_找_到_猫_。_我_会_继_续_搜索_其他_地_方_。____________
工具调用结束:
where_is_cat_hiding
{'input': {'idea': '在沙发底下'}, 'output': '在书架上吗？'}
_不_是_，在_沙_发_底_下_。_我_会_去_看_看_。___________
工具调用结束:
get_items
{'input': {'place': '沙发底下'}, 'output': '算了，找不到'}
_在_沙_发_底_下_也_没有_找_到_猫_。_让_我_再_想_想_其他_地_方_。_____________
工具调用结束:
where_is_cat_hiding
{'input': {'idea': '在厨房柜子里'}, 'output': '在床底吗？'}
_不_是_，在_厨_房_柜_子_里_。_我_会_去_看_看_。___________
工具调用结束:
get_items
{'input': {'place': '厨房柜子'}, 'output': '算了，找不到'}
_在_厨_房_柜_子_里_也_没有_找_到_猫_。_让_我_再_想_想_其他_地_方_。____________
工具调用结束:
where_is_cat_hiding
{'input': {'idea': '在花盆后面'}, 'output': '在书架上吗？'}
_不_是_，在_花_盆_后_面_。_我_会_去_看_看_。_________
工具调用结束:
get_items
{'input': {'place': '花盆'}, 'output': '算了，找不到'}
_在_花_盆_后_面_也_没有_找_到_猫_。_让_我_再_想_想_其他_地_方_。__________
工具调用结束:
where_is_cat_hiding
{'input': {

In [ ]:
events

## 更详细的流式日志

### astream_log

你可以通过 `include_names`/`include_tags`/`include_types` 来过滤想要包含的中间步骤日志，
也可以通过 `exclude_names`/`exclue_tags`/`exclude_types`来排除。

In [32]:
async for chunk in retrieval_chain.astream_log(
    "张朝阳在哪里上班？",
    version="v1",
    include_names=["本地检索", "LLM"],
    # include_tags=['Chroma'],
):
    print("-" * 80)
    print(chunk)

--------------------------------------------------------------------------------
RunLogPatch({'op': 'replace',
  'path': '',
  'value': {'final_output': None,
            'id': '6bcb8d1e-58ce-4d9f-80fb-ff9b3137637a',
            'logs': {},
            'name': 'RunnableSequence',
            'streamed_output': [],
            'type': 'chain'}})


Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


--------------------------------------------------------------------------------
RunLogPatch({'op': 'add',
  'path': '/logs/LLM',
  'value': {'end_time': None,
            'final_output': None,
            'id': '3f2907fd-3a58-4290-b1bf-09c57b0308bd',
            'metadata': {},
            'name': 'LLM',
            'start_time': '2024-02-15T11:44:49.534+00:00',
            'streamed_output': [],
            'streamed_output_str': [],
            'tags': ['seq:step:3'],
            'type': 'llm'}})
--------------------------------------------------------------------------------
RunLogPatch({'op': 'add', 'path': '/streamed_output/-', 'value': ''},
 {'op': 'replace', 'path': '/final_output', 'value': ''})
--------------------------------------------------------------------------------
RunLogPatch({'op': 'add', 'path': '/logs/LLM/streamed_output_str/-', 'value': ''},
 {'op': 'add',
  'path': '/logs/LLM/streamed_output/-',
  'value': AIMessageChunk(content='')})
--------------------------

### RunLogPatch

你也可以按照`RunLogPatch`对象所包含的`ops`属性中的JSON结构，来精细控制打印的日志内容。

In [42]:
import json

async for chunk in retrieval_chain.astream_log(
    "张朝阳在哪里上班？", version="v1", include_names=["本地检索", "LLM"]
):
    for s in chunk.ops:
        if(s['path'] == '/streamed_output/-'):
            print(s['value'], end="_", flush=True)
        else:
            pass

Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


_张_朝_阳_在_搜_狐_公司_上_班_。__